In [1]:
import os
import django
import requests
import datetime
import json
from tqdm import tqdm
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
# https://developers.themoviedb.org/3/discover/movie-discover
URL = 'https://api.themoviedb.org/3'
API_KEY = '164acb58532a315bea423c96031d8a71'

## 장르 DB

In [3]:
url_genres = f'{URL}/genre/movie/list?api_key=164acb58532a315bea423c96031d8a71'

In [4]:
result = requests.get(url_genres)

In [5]:
genres = result.json()['genres']

In [6]:
for g in genres:
    genre = Genre()
    genre.pk = g['id']
    genre.genre_name = g['name']
    genre.save()

## 영화 DB

In [7]:
def date_term(year):
    from_date = [f'{year}-{str(i).zfill(2)}-01' for i in range(1, 13)]
    to_date = [f'{year}-{str(i).zfill(2)}-31' for i in range(1, 13)]
    return from_date, to_date

In [8]:
def extend_movies(params):
    result = requests.get(f'{URL}/discover/movie', params=  params).json()
    movies.extend(result['results'])
    if params['page'] == 1:
        return result['total_pages']

In [9]:
for year in reversed(range(2015, 2018)):
    movies = []
    from_date, to_date = date_term(year)
    for month in reversed(range(12)):
        print(f'{year}년{month+1}월 수집중')
        params = {
            'api_key': API_KEY,
            'primary_release_date.gte': from_date[month],
            'primary_release_date.lte': to_date[month],
            'language': 'ko',
            'page': 1
        }
        total_pages = extend_movies(params)
        print(f'해당 월의 전체 페이지 수 : {total_pages}')
        for page_num in range(2, total_pages + 1):
            print(page_num, end='-')
            params['page'] = page_num
            extend_movies(params)
    with open(f'./movie_{year}.json', 'w') as fp:
        json.dump(movies, fp)

2017년12월 수집중
해당 월의 전체 페이지 수 : 88
2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-

KeyboardInterrupt: 

## Movies to DB

In [10]:
movies[0]

{'adult': False,
 'backdrop_path': '/wyv2Y9vXYJwJzF6cNVuVwVOsOUj.jpg',
 'genre_ids': [18, 14, 28],
 'id': 397567,
 'original_language': 'ko',
 'original_title': '신과함께-죄와 벌',
 'overview': '살인, 나태, 거짓, 불의, 배신, 폭력, 천륜 7개의 지옥에서 7번의 재판을 무사히 통과한 망자만이 환생하여 새로운 삶을 시작할 수 있다. 화재 사고 현장에서 여자아이를 구하고 죽음을 맞이한 소방관 자홍, 그의 앞에 저승차사 해원맥과 덕춘이 나타난다. 자신의 죽음이 아직 믿기지도 않는데 덕춘은 정의로운 망자이자 귀인이라며 그를 치켜세운다. 저승으로 가는 입구, 초군문에서 그를 기다리는 또 한 명의 차사 강림, 그는 차사들의 리더이자 앞으로 자홍이 겪어야 할 7개의 재판에서 변호를 맡아줄 변호사이기도 하다. 염라대왕에게 천년 동안 49명의 망자를 환생시키면 자신들 역시 인간으로 환생시켜 주겠다는 약속을 받은 삼차사들, 그들은 자신들이 변호하고 호위해야 하는 48번째 망자이자 19년 만에 나타난 의로운 귀인 자홍의 환생을 확신하지만, 각 지옥에서 자홍의 과거가 하나 둘씩 드러나면서 예상치 못한 고난과 맞닥뜨리는데…',
 'popularity': 89.743,
 'poster_path': '/5j2YVF7VouLG0Ze96SEsj4DnVQM.jpg',
 'release_date': '2017-12-14',
 'title': '신과함께-죄와 벌',
 'video': False,
 'vote_average': 8,
 'vote_count': 382}

In [11]:
with open(f'./movie_{2018}.json', 'r') as fp:
    movies = json.load(fp)

In [12]:
import warnings
warnings.filterwarnings(action='ignore')

In [13]:
# # using bulk
# to_db_movies = []

# for mv in movies[:10]:
#     movie = Movie()
#     movie.title_en = mv['original_title']
#     movie.title_ko = mv['title']
#     movie.rate = mv['vote_average']
#     movie.rate_people_count = mv['vote_count']
#     movie.poster_path = mv['poster_path']
#     movie.description = mv['overview']
#     movie.release_date = datetime.datetime.strptime(mv['release_date'], '%Y-%m-%d')
#     to_db_movies.append(movie)

# Movie.objects.bulk_create(to_db_movies)

In [14]:
with open(f'./movie_{2021}.json', 'r') as fp:
    movies = json.load(fp)

In [15]:
for mv in tqdm(movies[10:]):
    movie = Movie()
    movie.pk = mv['id']
    movie.title_en = mv['original_title']
    movie.title_ko = mv['title']
    movie.rate = mv['vote_average']
    movie.rate_people_count = mv['vote_count']
    movie.poster_path = mv['poster_path']
    movie.description = mv['overview']
    movie.release_date = datetime.datetime.strptime(mv['release_date'], '%Y-%m-%d')
    movie.save()
    for gid in mv['genre_ids']:
        movie.genres.add(Genre.objects.get(pk=gid))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8793/8793 [00:40<00:00, 215.89it/s]
